# CISC/CMPE 452/COGS 400 Assignment 3 - Unsupervised Learning (10 points)  

Please put your name and student id

    Sam Alexander, #20158924

- Make sure to run all the cells from the beginning before submission. Do not clear out the outputs. You will only get credit for code that has been run.
- Mark will be deducted based on late policy (-1% of the course total marks per day after due date until the end date after which no assignments will be accepted).
- You can only use Numpy to build the models. Other packages such as Pandas, Sklearn and Scipy can be used for evaluation metrics calculating, data processing, and file reading and writing.

### Files need to be uploaded for this assignment: A3.ipynb, output.wav, and output.csv

## Part 1 Principle Component Analysis Network (5 points)
The dataset "data/sound.csv" contains two sounds recorded by the two microphones. The goal of this assignment is using PCA network to find the approximation of the first principal component.
- Build a PCA network (refer to Principal Component Analysis slide #22 and #23) to reduce the number of features from 2 to 1 (3 points)  
- Train the model and generate the processed data (1 point)  
- Save the data into output.wav and output.csv files (1 point)  
- Compare the sound_o.wav (audio with noise) and output.wav (audio is denoised)  

In [3]:
import numpy as np
import pandas as pd
from scipy.io import wavfile

In [4]:
samrate = 8000

In [5]:
# read csv into array
txtData = np.genfromtxt('data/sound.csv', delimiter=',')
txtData.shape

(50000, 2)

In [6]:
# save array to WAV file
scaledData = np.int16(txtData * samrate)
wavfile.write('data/sound_o.wav', samrate, scaledData)

In [7]:
# read WAV file into array
# The data in sound.csv is processed
# If you use the data generated here, you need to process the data by adding wavData = wavData / samrate
samrate, wavData = wavfile.read('data/sound_o.wav')
samrate, wavData.shape

(8000, (50000, 2))

In [8]:
# save array to csv file
np.savetxt('data/sound_o.csv', txtData, delimiter=',')

In [9]:
# build PCA model and only Numpy can be used
class PCA(object):
    def __init__(self, lr, epoch):
        self.learning_rate = lr
        self.epoch = epoch
        
    def train(self, x, n_components):
        #normalize the data
        mean = np.mean(x, axis=0)
        norm_data = x - mean
        
        num_rows, num_cols = np.shape(x)
        
        #initialize weights
        self.W = np.random.rand(num_cols, 1)*0.01
        learning_rate = self.learning_rate
        
        for i in range(self.epoch):
            #check each x input and adjust
            for x_input in norm_data:
                #calculate activation
                output_y = np.matmul(x_input, self.W)
            
                #calculate weight adjustment term
                first_term = np.dot(np.sum(output_y), x_input)
                second_term = (output_y**2)*np.transpose(self.W)
                delta_w = learning_rate*(first_term - second_term)
            
                #apply adjustment
                self.W += np.transpose(delta_w)
            
    #this function will return a predicted single-output array from a N-size input array  
    def predict(self, x):
        output = np.matmul(x, self.W)
        output_mean = np.mean(output, axis = 0)
        output -= output_mean
        output[0] = 0 #set first input as zero
        return output

In [10]:
# initialize and train the model
pca = PCA(0.05, 5)
pca.train(txtData, len(txtData[:,]))
output = pca.predict(txtData)

In [11]:
# save the data
wav_output = samrate*output
wavfile.write('data/output.wav', samrate, np.int16(wav_output))
np.savetxt('data/output.csv', output, delimiter=',')

## Part 2 K-Means Clustering Algorithm (5 points)
The dataset is [Palmer Archipelago (Antarctica) penguin data](https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data) which has 6 features and 1 label called species (Chinstrap, Adélie, or Gentoo)  
The dataset is saved in the "data/penguins_size.csv" file and preprocessed into x_train, x_test, y_train, y_test  
- Build a K-Means clustering algorithm (refer to Unsupervised Learning slide #29) to cluster the preprocessed data (2 points)  
- Standardize the data and train the model with the training set (1 point)  
- Evaluate the model and print the confusion matrixes with both training and test sets (2 points)  


In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy.linalg import norm
from sklearn import metrics

In [75]:
# load the dataset
data = pd.read_csv('data/penguins_size.csv')
data.head()

species     island  culmen_length_mm  culmen_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen              39.1             18.7              181.0   
1  Adelie  Torgersen              39.5             17.4              186.0   
2  Adelie  Torgersen              40.3             18.0              195.0   
3  Adelie  Torgersen               NaN              NaN                NaN   
4  Adelie  Torgersen              36.7             19.3              193.0   

   body_mass_g     sex  
0       3750.0    MALE  
1       3800.0  FEMALE  
2       3250.0  FEMALE  
3          NaN     NaN  
4       3450.0  FEMALE

In [76]:
# data preprocessing
data = data.dropna()
data = data[data['sex'] != '.']

cleanup_nums = {"species": {"Adelie": 0, "Chinstrap": 1, "Gentoo": 2},
                "island": {"Biscoe": 0, "Dream": 1, "Torgersen": 2},
                "sex": {"MALE": 0.0, "FEMALE": 1.0}}
data = data.replace(cleanup_nums)

data.head()

species  island  culmen_length_mm  culmen_depth_mm  flipper_length_mm  \
0        0       2              39.1             18.7              181.0   
1        0       2              39.5             17.4              186.0   
2        0       2              40.3             18.0              195.0   
4        0       2              36.7             19.3              193.0   
5        0       2              39.3             20.6              190.0   

   body_mass_g  sex  
0       3750.0  0.0  
1       3800.0  1.0  
2       3250.0  1.0  
4       3450.0  1.0  
5       3650.0  0.0

In [77]:
x = np.array(data.drop(['species'], axis=1).copy())
y = np.array(data['species'].copy()).astype(int)

In [78]:
# data standardization
#x = (x-x.mean(axis = 0))/x.std(axis = 0)
#data normalization (you need this to be able to do the dot product to determine distance)
x = (x-np.ndarray.min(x, axis = 0))/(np.ndarray.max(x, axis = 0)-np.ndarray.min(x, axis = 0))

In [79]:
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((266, 6), (67, 6), (266,), (67,))

In [80]:
# calculate the confusion matrix
def evaluator(y, y_pred):    
    confusion_matrix = metrics.confusion_matrix(y, y_pred)
    print('Confusion matrix:\n', confusion_matrix)

In [88]:
# setup a baseline model
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3) # n_clusters - the number of clusters
km.fit(x_train)
y_pred = km.predict(x_train)
evaluator(y_train, y_pred)
y_pred = km.predict(x_test)
evaluator(y_test, y_pred)

Confusion matrix:
 [[47 60  0]
 [26 32  0]
 [51  0 50]]
Confusion matrix:
 [[26 13  0]
 [ 8  2  0]
 [ 7  0 11]]


In [117]:
# build K-means model and only Numpy can be used
class KMean(object):
    def init(self, n_clusters):
        self.n_clusters = 3

    def create_cents(self, x):
        retval = np.zeros((self.n_clusters, x.shape[1]))
        for k in range(self.n_clusters):
            retval[k,:] = x[np.random.choice(len(x)),:]
        return retval

    def compute_new_centroids(self, x, labels):
        centroids = np.zeros(((self.n_clusters), x.shape[1]))

        for k in range(self.n_clusters):
            centroids[k, :] = np.mean(x[labels == k, :], axis = 0)
        return centroids

    def compute_dist(self, x, cents):
        distance = np.zeros((x.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            row_norm = norm(x - cents[k,:], axis = 1)
            distance[:, k] = np.square(row_norm)
        return distance

    def find_closest(self, distance):
        return np.argmin(distance, axis = 1)

    def compute_error(self, x, labels, centroids):
        distance = np.zeros(x.shape[0])

        for k in range(self.n_clusters):
            distance[labels == k] = norm(x[labels == k] - centroids[k], axis = 1)
        return np.sum(np.square(distance))

    def train(self, x, y, x_test, y_test, n_iters):
        self.n_clusters = 3
        self.W = self.create_cents(x)
        
        #for checking accuracy later
        self.history = {}
        self.history['error'], self.history['train_acc'], self.history['test_acc'] = [], [], []
        
        for i in range(n_iters):
            old_centroids = self.W
            distance = self.compute_dist(x,old_centroids)
            self.labels = self.find_closest(distance)
            self.W = self.compute_new_centroids(x, self.labels)
            
            if np.all(old_centroids == self.W):
                break
            self.error = self.compute_error(x,self.labels,self.W)
            
            #now print accuracy/error per epoch
            
            self.history['error'].append(self.error)
            
            train_acc = (self.predict(x) == y).sum() / y.shape[0]
            self.history['train_acc'].append(train_acc)

            test_acc = (self.predict(x_test) == y_test).sum() / y_test.shape[0]
            self.history['test_acc'].append(test_acc)
            
            print('epoch %d, train error %.4f, train acc %.4f, test acc %.4f' % (i+1, self.error, train_acc, test_acc))

    def predict(self,x):
        distance = self.compute_dist(x, self.W)
        return self.find_closest(distance)

In [126]:
# initialize and train the model
k = KMean()
k.train(x_train,y_train,x_test,y_test,20)

epoch 1, train error 79.9137, train acc 0.4361, test acc 0.4179
epoch 2, train error 71.1050, train acc 0.4436, test acc 0.4478
epoch 3, train error 70.3819, train acc 0.4436, test acc 0.4478
epoch 4, train error 70.1551, train acc 0.4361, test acc 0.4478
epoch 5, train error 70.0501, train acc 0.4286, test acc 0.4478
epoch 6, train error 69.9777, train acc 0.4211, test acc 0.4030
epoch 7, train error 69.5896, train acc 0.4173, test acc 0.4030
epoch 8, train error 64.2707, train acc 0.4474, test acc 0.4776
epoch 9, train error 57.6504, train acc 0.4850, test acc 0.5821
epoch 10, train error 52.0449, train acc 0.4850, test acc 0.5821
epoch 11, train error 51.9193, train acc 0.4850, test acc 0.5821


In [127]:
# evaluate the model and print the confusion matrixes for both training and test sets
evaluator(y_train, k.predict(x_train))
evaluator(y_test, k.predict(x_test))

Confusion matrix:
 [[47 60  0]
 [26 32  0]
 [51  0 50]]
Confusion matrix:
 [[26 13  0]
 [ 8  2  0]
 [ 7  0 11]]
